In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_path = "../data-unversioned/p1ch7"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))
])

cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_test  = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

cifar10_trainX = torch.stack([img.view(-1) for img, _ in cifar10_train])
cifar10_trainY = torch.tensor([label for _, label in cifar10_train], dtype=torch.long)

cifar10_testX = torch.stack([img.view(-1) for img, _ in cifar10_test])
cifar10_testY = torch.tensor([label for _, label in cifar10_test], dtype=torch.long)

cifar10_trainX = cifar10_trainX.to(device)
cifar10_trainY = cifar10_trainY.to(device)
cifar10_testX  = cifar10_testX.to(device)
cifar10_testY  = cifar10_testY.to(device)

train_dataset = torch.utils.data.TensorDataset(cifar10_trainX, cifar10_trainY)
val_dataset   = torch.utils.data.TensorDataset(cifar10_testX, cifar10_testY)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=1000, shuffle=False)

In [ ]:
input_size = 3 * 32 * 32
model = nn.Sequential(
    nn.Linear(input_size, 1024),
    nn.Tanh(),
    nn.Linear(1024, 512),
    nn.Tanh(),
    nn.Linear(512, 128),
    nn.Tanh(),
    nn.Linear(128, 10),
    nn.LogSoftmax(dim=1)
).to(device)

learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()

n_epochs = 300
start = time.time()

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_Y in train_loader:
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

end = time.time()
print("\nTraining Time =", end - start, "seconds")
print("Final Training Loss =", epoch_loss)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_X, batch_Y in val_loader:
        outputs = model(batch_X)
        preds = outputs.argmax(dim=1)
        correct += (preds == batch_Y).sum().item()
        total += batch_Y.size(0)

accuracy = correct / total
print("Evaluation Accuracy =", accuracy)

Epoch 1, Loss: 2.1704
Epoch 2, Loss: 2.0347
Epoch 3, Loss: 1.9674
Epoch 4, Loss: 1.9233
Epoch 5, Loss: 1.8910
Epoch 6, Loss: 1.8657
Epoch 7, Loss: 1.8448
Epoch 8, Loss: 1.8268
Epoch 9, Loss: 1.8109
Epoch 10, Loss: 1.7967
Epoch 11, Loss: 1.7836
Epoch 12, Loss: 1.7717
Epoch 13, Loss: 1.7604
Epoch 14, Loss: 1.7499
Epoch 15, Loss: 1.7401
Epoch 16, Loss: 1.7307
Epoch 17, Loss: 1.7219
Epoch 18, Loss: 1.7137
Epoch 19, Loss: 1.7057
Epoch 20, Loss: 1.6980
Epoch 21, Loss: 1.6909
Epoch 22, Loss: 1.6838
Epoch 23, Loss: 1.6771
Epoch 24, Loss: 1.6706
Epoch 25, Loss: 1.6642
Epoch 26, Loss: 1.6582
Epoch 27, Loss: 1.6520
Epoch 28, Loss: 1.6462
Epoch 29, Loss: 1.6403
Epoch 30, Loss: 1.6347
Epoch 31, Loss: 1.6290
Epoch 32, Loss: 1.6235
Epoch 33, Loss: 1.6181
Epoch 34, Loss: 1.6127
Epoch 35, Loss: 1.6073
Epoch 36, Loss: 1.6020
Epoch 37, Loss: 1.5965
Epoch 38, Loss: 1.5912
Epoch 39, Loss: 1.5857
Epoch 40, Loss: 1.5807
Epoch 41, Loss: 1.5753
Epoch 42, Loss: 1.5698
Epoch 43, Loss: 1.5645
Epoch 44, Loss: 1.55